In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, read_pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from environment import training_columns_regex
import interpret
import matplotlib.pyplot as plt
from interpret.glassbox import ExplainableBoostingClassifier
from interpret import show
from interpret.blackbox import ShapKernel, LimeTabular, MorrisSensitivity, PartialDependence
from interpret import set_visualize_provider
from interpret.provider import InlineProvider
set_visualize_provider(InlineProvider())

In [7]:
!pip3 install interpret 


[notice] A new release of pip is available: 23.0.1 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
conda install -c conda-forge interpret

Retrieving notices: ...working... done
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\ProgramData\anaconda3

  added / updated specs:
    - interpret


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2023.7.22  |       h56e8100_0         146 KB  conda-forge
    certifi-2023.7.22          |     pyhd8ed1ab_0         150 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         297 KB

The following packages will be UPDATED:

  ca-certificates                       2023.5.7-h56e8100_0 --> 2023.7.22-h56e8100_0 
  certifi                             2023.5.7-pyhd8ed1ab_0 --> 2023.7.22-pyhd8ed1ab_0 




ca-certificates-2023 | 146 KB    |            |   0% 

certifi-2023.7.22    | 150 KB    |            |   0% 

certifi-2023.7.22    | 150 KB    | #          |


EnvironmentNotWritableError: The current user does not have write permissions to the target environment.
  environment location: C:\ProgramData\anaconda3




In [5]:
df_path=r"C:\Users\Ahmed Guebsi\Desktop\Data_test\.clean_raw_df_adhd.pkl"
df: DataFrame = read_pickle(df_path)
# Convert complex values to real
df_real = df.applymap(lambda x: x.real)

#print(isnull_any(df))
#glimpse_df(df)
#print(rows_with_null(df))

seed = 42
np.random.seed(seed)

training_columns = list(df.iloc[:, df.columns.str.contains(training_columns_regex)].columns)

# Remove rows with null values
df_cleaned = df.dropna().reset_index(drop=True)

X = df_cleaned.drop("is_adhd", axis=1)
y = df_cleaned.loc[:, "is_adhd"]
print(y)
print(df_cleaned.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)

ebm = ExplainableBoostingClassifier()
ebm.fit(X_train, y_train)
auc = roc_auc_score(y_test, ebm.predict_proba(X_test)[:, 1])
print("AUC: {:.3f}".format(auc))

pca = PCA()
rf = RandomForestClassifier(random_state=seed)

blackbox_model = Pipeline([('pca', pca), ('rf', rf)])
blackbox_model.fit(X_train, y_train)

lime = LimeTabular(blackbox_model, X_train)
show(lime.explain_local(X_test[:5], y_test[:5]), 0)

plt.show()

pdp = PartialDependence(blackbox_model, X_train)
show(pdp.explain_global(), 0)

msa = MorrisSensitivity(blackbox_model, X_train)
show(msa.explain_global())

shap = ShapKernel(blackbox_model, X_train)
shap_local = shap.explain_local(X_test[:5], y_test[:5])

show(shap_local, 0)

0       0
1       0
2       0
3       0
4       0
       ..
8141    1
8142    1
8143    1
8144    1
8145    1
Name: is_adhd, Length: 8146, dtype: int32
(8146, 801)
AUC: 0.925


ImportError: cannot import name 'get_current_traceback' from 'werkzeug.debug.tbtools' (C:\ProgramData\anaconda3\Lib\site-packages\werkzeug\debug\tbtools.py)